In [16]:
results_dir = 'results_tmp'
import importlib
import matplotlib.pyplot as plt
import seaborn as sns
import os
os.makedirs(results_dir, exist_ok=True)

In [2]:
from experiment_info import samples, data_dir, puffs, params

import tmp_activity_traces as activity_traces
import tmp_video_IO as video_IO
import tmp_binary_mask as binary_mask

2024-07-02 11:21:38.614992: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-02 11:21:38.615044: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-02 11:21:38.616305: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [3]:
# import custom functions
import functions as fn

print(f'data directory: {data_dir}')
print(f'Number of samples: {len(samples)}')
num_odors = len(puffs)

print(f'Number of odors: {num_odors}')
print(f'x,y,z dimensions:', params['x_dim'], params['y_dim'], params['z_dim'])
print(f'Number of frames to analyze:', params['n_frames_to_analyze'])
print(f'Number of initial frames for df/f normalization:', params['background_frames'])

data directory: /mnt/cup/labs/mcbride/bjarnold/new_analysis/data/Mar_22_2024/1_RegisteredBrains
Number of samples: 15
Number of odors: 72
x,y,z dimensions: 128 128 24
Number of frames to analyze: 112
Number of initial frames for df/f normalization: 20


In [4]:

import glob
import pandas as pd
from collections import defaultdict
import pickle

In [5]:
class Brain:

    # This class stores data; methods have been put in modules for re-usability
    #
    # name: name of the sample
    # vid_fnames: the file paths of all the individual videos
    # video: the full video, a concatenation of all individual videos
    # binary_mask: 3D outline of the antennal lobe, region in video of higher pixel intensities according to otsu thresholding
    # binary_mask_frac: the fraction of pixels contained within binary_mask, compared to the entire 3d volume
    # mean_activity: the mean pixel intensity wtihin binary mask, over time; this is df_f if videos have been normalized against background activity
    # mean_activity_paraffin_subtracted: same as mean_activity except the signal of the paraffin odor has been subtracted
    # maxs_per_odor: a dictionary where keys are names of odors, values are lists of size 2, one for each trial in which the odor was administered
    # aucs_per_odor: same as maxs_per_odor, except calculating the area under the curve

    def __init__(self, name, vid_fnames):
        self.name = name
        self.vid_fnames = vid_fnames

    def __repr__(self):
        # return f"Brain for {self.name} containing {len(self.vid_fnames)} videos"
        # Generate a comma-separated string of attribute names
        attribute_keys = ', '.join(self.__dict__.keys())
        return f"{self.__class__.__name__}({attribute_keys})"

In [6]:
# the brains list will store instances of the Brain class that will keep track of data and preprocessing methods
brains = []

for samp in samples:
    v = glob.glob(f"{data_dir}/{samp}/*.registered.tif")
    v = sorted(v)
    num_vids = len(v)
    assert num_vids == len(puffs), f"I found {num_vids} videos for sample {samp}, but there are {len(puffs)} odors. The number of videos and odors should be equivalent."
    
    brain = Brain(samp, v)
    brains.append(brain)


In [7]:
importlib.reload(video_IO)

for i,b in enumerate(brains):
    if i != 0:
        continue
    # using filenames of videos, load them into movie objects, creating video instance attribute
    b.video = video_IO.load_videos(b.vid_fnames, params, normalize=False)

In [8]:
for i,b in enumerate(brains):
    if i != 0:
        continue
    # use otsu thresholding to find binary mask, specify a results dir to save the binary mask object and plot
    b.binary_mask, b.binary_mask_frac = binary_mask.find_binary_mask(b.video, params)
    binary_mask.save_and_plot_binary_mask(b.name, b.binary_mask, params, results_dir)

    # use the binary mask to compute mean activity over time
    # except here we will re-load the videos, overwriting self.video, and normalize them against the first params['n_frames_to_analyze']
    b.video = video_IO.load_videos(b.vid_fnames, params, normalize=True)
    
    b.mean_activity = binary_mask.mean_activity_within_binary_mask(b.video, b.binary_mask, params)
    # b.subtract_paraffin(puffs, params)


    

(393216,) (8064, 393216)


In [9]:
importlib.reload(activity_traces)

for i,b in enumerate(brains):
    if i != 0:
        continue
    b.maxs_per_odor, b.argmaxs_per_odor = activity_traces.max_activity_per_odor(b.mean_activity, puffs, params)
    b.aucs_per_odor = activity_traces.activity_auc_per_odor(b.mean_activity, puffs, params)
    
    b.mean_activity_paraffin_subtracted = activity_traces.subtract_paraffin_response(b.mean_activity, puffs, params)


In [10]:
# for i,b in enumerate(brains):
#     if i != 0:
#         continue
#     b.compute_max_responses(puffs, params)
#     b.compute_AUC(puffs, params)

## save the mean trace activity as csv file

In [11]:
mean_activity = {b.name : b.mean_activity for i,b in enumerate(brains) if i == 0}
mean_activity_df = pd.DataFrame.from_dict(mean_activity)
mean_activity_df.to_csv(f'{results_dir}/mean_activity_within_mask.csv', index=False)

## save maxs per odor as dataFrame

In [12]:
def convert_to_df(brains, puffs, metric):
    df_list = []
    for i,b in enumerate(brains):
        if i != 0:
            continue
        df_tmp = pd.DataFrame.from_dict(getattr(b, metric))
        df_tmp['samp'] = b.name
        df_tmp['subpop'] = b.name.split('_')[1]
        df_tmp['trial'] = df_tmp.index+1
        df_list.append(df_tmp)
    df = pd.concat(df_list)
    df = df.reset_index(drop=True)
    df = pd.melt(df, id_vars=['samp', 'subpop', 'trial'], var_name='odor', value_name='value')

    odor_order = {}
    for puff in puffs:
        if puff.trial == 1:
            odor_order[puff.odor_name] = puff.number

    df['odor_order'] = df['odor'].map(odor_order)
    return df

peak_max_df = convert_to_df(brains, puffs, 'maxs_per_odor')
peak_auc_df = convert_to_df(brains, puffs, 'aucs_per_odor')
peak_max_df


,samp,subpop,trial,odor,value,odor_order
0,230913_ORL_GCaMP6f_F1,ORL,1,hexanal4.375,0.000476,0
1,230913_ORL_GCaMP6f_F1,ORL,2,hexanal4.375,0.000475,0
2,230913_ORL_GCaMP6f_F1,ORL,1,hexanal4.03,0.001123,1
3,230913_ORL_GCaMP6f_F1,ORL,2,hexanal4.03,0.000477,1
4,230913_ORL_GCaMP6f_F1,ORL,1,hexanal2.825,0.001073,2
...,...,...,...,...,...,...
67,230913_ORL_GCaMP6f_F1,ORL,2,hexanoic acid,0.001025,33
68,230913_ORL_GCaMP6f_F1,ORL,1,camphor,0.009764,34
69,230913_ORL_GCaMP6f_F1,ORL,2,camphor,0.006896,34
70,230913_ORL_GCaMP6f_F1,ORL,1,1-octen-3-ol,0.000644,35


In [17]:
def make_mean_activity_plot(brains, params, metric):

    fig, axs = plt.subplots(1, 1, figsize=(16, 4))

    for i,b in enumerate(brains):
        if i != 0:
            continue

        activity = getattr(b, metric)
        plt.plot(activity + i*0.02, c='black')  # Offset each trace by i*3
        # print sample name on the right
        plt.text(len(activity)*1.02, i*0.02, b.name, color='black')

    # print the names of the odors on the x-axis
    odor_names = []
    positions = []
    for i,puff in enumerate(puffs):
        odor_names.append(puff.odor_name)
        positions.append(i*params['n_frames_to_analyze'] + params['n_frames_to_analyze']/2)
    plt.xticks(positions, odor_names, rotation=90)

    # draw vertical lines to separate odors
    for i in range(len(puffs)):
        plt.axvline((i+1)*params['n_frames_to_analyze'], color="black", linestyle="--", alpha=0.1)

    plt.yticks([])
    # supress grid lines
    plt.grid(False)
    sns.despine()

    plt.savefig(f'{results_dir}/{metric}.png', dpi=300)
    plt.close()

make_mean_activity_plot(brains, params, metric='mean_activity')
make_mean_activity_plot(brains, params, metric='mean_activity_paraffin_subtracted')